## This notebook demonstrates how to parse HAL logs

In [ ]:
import os

import storm_control.sc_library.log_timing as logTiming

os.chdir("/home/hbabcock/Data/storm_control/logs/")
print(os.getcwd())


In [ ]:
# Note that the no extensions are included in the log name.
all_msgs = logTiming.logTiming("hal4000_4")

### How much total time?

In [ ]:
print("{0:.3f} seconds".format(logTiming.processingTime(all_msgs)))

### How much time was spent on TCP message?

In [ ]:
msgs = logTiming.groupBySource(all_msgs)["tcp_control"]
print("{0:.3f} total seconds".format(logTiming.processingTime(msgs)))
print()

grp = logTiming.groupByMsgType(msgs)
for key in sorted(grp):
    tmp = grp[key]
    print("'" + key + "', {0:0d} counts, {1:.3f} seconds".format(len(tmp), logTiming.processingTime(tmp)))
    

### Much time was spend on film messages?

In [ ]:
msgs = logTiming.groupBySource(all_msgs)["film"]
print("{0:.3f} total seconds".format(logTiming.processingTime(msgs)))
print()

grp = logTiming.groupByMsgType(msgs)
for key in sorted(grp):
    tmp = grp[key]
    print("'" + key + "', {0:0d} counts, {1:.3f} seconds".format(len(tmp), logTiming.processingTime(tmp)))
    

### How much time was spent on each message?

In [ ]:
grp = logTiming.groupByMsgType(all_msgs)
for key in sorted(grp):
    tmp = grp[key]
    print("'" + key + "', {0:0d} counts, {1:.3f} seconds".format(len(tmp), logTiming.processingTime(tmp)))
    

### How much time was spent filming?

In [ ]:
# Get the 'film lockout' messages.
fl = logTiming.groupByMsgType(all_msgs)["film lockout"]

# Sort by starting time, reversed.
fl = sorted(fl, key = lambda x: x.getCreatedTime(), reverse = True)

# We assume that the last 'film lockout' message was at the end of a film.
total_x = 0
total_time = 0.0
for i in range(0, len(fl), 2):
    if ((i+1)==len(fl)):
        continue
    total_x += 1
    total_time += fl[i].getCreatedTime() - fl[i+1].getCreatedTime()
    
print("{0:0d} films, {1:.3f} sec/film".format(total_x, total_time/total_x))

### How much dead time between films?

In [ ]:
# Get the 'film lockout' messages.
fl = logTiming.groupByMsgType(all_msgs)["film lockout"]

# Sort by starting time, reversed.
fl = sorted(fl, key = lambda x: x.getCreatedTime(), reverse = True)

# We assume that the last 'film lockout' message was at the end of a film.
total_x = 0
total_time = 0.0
for i in range(1, len(fl), 2):
    if ((i+1)==len(fl)):
        continue    
    total_x += 1
    total_time += fl[i].getCreatedTime() - fl[i+1].getCreatedTime()
    
print("{0:0d} breaks, {1:.3f} sec/break".format(total_x, total_time/total_x))